In [13]:
from datasets import load_dataset, DatasetDict

dataset_id = DatasetDict()
dataset_id['train'] = load_dataset("mozilla-foundation/common_voice_11_0", "id", split="train+validation")
dataset_id['test'] = load_dataset("mozilla-foundation/common_voice_11_0", "id", split="test")
# dataset_id['train'] = dataset_id['train'].select(range(6000))
dataset_id['test'] = dataset_id['test'].select(range(200))
dataset_id

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 8274
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 200
    })
})

In [14]:
dataset_id = dataset_id.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
dataset_id

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 8274
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 200
    })
})

In [15]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="id", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

input_str = dataset_id["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Input:                 halo dunia!
Decoded w/ special:    <|startoftranscript|><|id|><|transcribe|><|notimestamps|>halo dunia!<|endoftext|>
Decoded w/out special: halo dunia!
Are equal:             True


In [16]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="id", task="transcribe")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
dataset_id["train"][0]

{'audio': {'path': '/home/jupyter/.cache/huggingface/datasets/downloads/extracted/d160bf91a3bdf59a0c4db9adfec4d6b50be9948cc406a4c3a2d810779be58081/id_train_0/common_voice_id_25361422.mp3',
  'array': array([ 0.00000000e+00, -5.75539616e-13, -8.17124146e-13, ...,
          1.40947872e-04,  1.74419692e-05, -1.29718828e-05]),
  'sampling_rate': 48000},
 'sentence': 'halo dunia!'}

In [18]:
from datasets import Audio

dataset_id = dataset_id.cast_column("audio", Audio(sampling_rate=16000))

In [19]:
dataset_id["train"][0]

{'audio': {'path': '/home/jupyter/.cache/huggingface/datasets/downloads/extracted/d160bf91a3bdf59a0c4db9adfec4d6b50be9948cc406a4c3a2d810779be58081/id_train_0/common_voice_id_25361422.mp3',
  'array': array([-2.32830644e-10,  0.00000000e+00,  6.98491931e-10, ...,
          2.50757148e-04,  2.26852426e-04,  1.24129729e-04]),
  'sampling_rate': 16000},
 'sentence': 'halo dunia!'}

In [20]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

dataset_id = dataset_id.map(prepare_dataset, remove_columns=dataset_id.column_names["train"])


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
from transformers import WhisperForConditionalGeneration
import torch

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "id"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda


In [22]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [23]:
import evaluate

metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./whisper-small-indo",  
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=0.000001,
    warmup_steps=100,
    max_steps=1000,
    fp16=True,
    gradient_checkpointing=True,
    save_steps=1000,
    logging_steps=200,
    report_to=["tensorboard"],
    push_to_hub=True,
)

trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=dataset_id["train"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)

trainer.train()


max_steps is given, it will override any value given in num_train_epochs
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
200,0.788400
400,0.423300
600,0.442200
800,0.403700
1000,0.393500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


TrainOutput(global_step=1000, training_loss=0.4902207946777344, metrics={'train_runtime': 1310.5035, 'train_samples_per_second': 3.052, 'train_steps_per_second': 0.763, 'total_flos': 1.15434160128e+18, 'train_loss': 0.4902207946777344, 'epoch': 0.4833252779120348})